## DEEPER MULTILAYER PERCEPTRON WITH DROPOUT

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
%matplotlib inline

# LOAD MNIST

In [2]:
mnist = input_data.read_data_sets('data/', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Instructions for updating:
Please write your own downloading logic.


Instructions for updating:
Please use tf.data to implement this functionality.


Extracting data/train-images-idx3-ubyte.gz


Instructions for updating:
Please use tf.data to implement this functionality.


Extracting data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.


Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


# DEFINE MODEL

In [3]:
# NETWORK TOPOLOGIES
n_input    = 784 # MNIST data input (img shape: 28*28)
n_hidden_1 = 512 # 1st layer num features
n_hidden_2 = 512 # 2nd layer num features
n_hidden_3 = 256 # 3rd layer num features
n_classes  = 10 # MNIST total classes (0-9 digits)
# INPUT AND OUTPUT
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])
dropout_keep_prob = tf.placeholder("float")
# VARIABLES
stddev = 0.05
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1], stddev=stddev)),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2], stddev=stddev)),
    'h3': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_3], stddev=stddev)),
    'out': tf.Variable(tf.random_normal([n_hidden_3, n_classes], stddev=stddev))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'b3': tf.Variable(tf.random_normal([n_hidden_3])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}
print ("NETWORK READY")

NETWORK READY


In [7]:
def multilayer_perceptron(_X, _weights, _biases, _keep_prob):
    x_1 = tf.nn.relu(tf.add(tf.matmul(_X, _weights['h1']), _biases['b1']))
    layer_1 = x_1
    x_2 = tf.nn.relu(tf.add(tf.matmul(layer_1, _weights['h2']), _biases['b2']))
    layer_2 = x_2
    x_3 = tf.nn.relu(tf.add(tf.matmul(layer_2, _weights['h3']), _biases['b3']))
    layer_3 = tf.nn.dropout(x_3, _keep_prob) ##对全连接层采用dropout随机丢失一些特征，此方法能够防止过拟合
    ##在预测的时候，不会丢弃任何特征，整体的进行预测，所以假设某个变量上一次没有被丢弃，这一次被丢弃了，只是训练这一次的时候没有将它计算进损失中去，但是还不清楚有没有去更新他
    return (tf.matmul(layer_3, _weights['out']) + _biases['out'])

# DEFINE FUNCTIONS

In [9]:
# PREDICTION
pred = multilayer_perceptron(x, weights, biases, dropout_keep_prob)

# LOSS AND OPTIMIZER
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=pred)) ##此处必须要按照格式写，加上labels 跟logits 不然会报错
optm = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost) 
corr = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))    
accr = tf.reduce_mean(tf.cast(corr, "float"))

# INITIALIZER
init = tf.initialize_all_variables()
print ("FUNCTIONS READY")

Instructions for updating:
Use `tf.global_variables_initializer` instead.


FUNCTIONS READY


# RUN

In [10]:
# PARAMETERS
training_epochs = 20
batch_size      = 100
display_step    = 4
# LAUNCH THE GRAPH
sess = tf.Session()
sess.run(init)
# OPTIMIZE
for epoch in range(training_epochs):
    avg_cost = 0.
    total_batch = int(mnist.train.num_examples/batch_size)
    # ITERATION
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feeds = {x: batch_xs, y: batch_ys, dropout_keep_prob: 0.6}
        sess.run(optm, feed_dict=feeds)##在进行梯度下降的时候没有进行损失的计算
        feeds = {x: batch_xs, y: batch_ys, dropout_keep_prob: 1.0}
        avg_cost += sess.run(cost, feed_dict=feeds)##在计算整体的损失的时候加入了所有的特征
    avg_cost = avg_cost / total_batch
    # DISPLAY
    if (epoch+1) % display_step == 0:
        print ("Epoch: %03d/%03d cost: %.9f" % (epoch, training_epochs, avg_cost))
        feeds = {x: batch_xs, y: batch_ys, dropout_keep_prob: 1.0}##在预测的时候，不会丢弃任何特征，整体的进行预测，所以假设某个变量上一次没有被丢弃，这一次被丢弃了，只是训练这一次的时候没有将它计算进损失中去，但是还不清楚有没有去更新他
        train_acc = sess.run(accr, feed_dict=feeds)
        print ("TRAIN ACCURACY: %.3f" % (train_acc))
        feeds = {x: mnist.test.images, y: mnist.test.labels, dropout_keep_prob: 1.0}
        test_acc = sess.run(accr, feed_dict=feeds)
        print ("TEST ACCURACY: %.3f" % (test_acc))
print ("OPTIMIZATION FINISHED")

Epoch: 003/020 cost: 0.035113383
TRAIN ACCURACY: 0.990


TEST ACCURACY: 0.974


Epoch: 007/020 cost: 0.009835481
TRAIN ACCURACY: 1.000
TEST ACCURACY: 0.978

Epoch: 011/020 cost: 0.005452721
TRAIN ACCURACY: 1.000
TEST ACCURACY: 0.982

Epoch: 015/020 cost: 0.003393464
TRAIN ACCURACY: 1.000
TEST ACCURACY: 0.980


Epoch: 019/020 cost: 0.003221291
TRAIN ACCURACY: 1.000
TEST ACCURACY: 0.982


OPTIMIZATION FINISHED
